# SGD Classifier
---
- LogisticRegression + SGD(확률적경사하강법) 적용한 분류
- 필수 파라미터 => 손실함수 설정

In [1]:
# 모듈 로딩
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

In [8]:
# 경고 무시 ---------------------------------
# 향후 deprecated 된다는 경고 출력 막음
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
# 파일 관련
FILE='../Data1/fish.csv'

## [1] 데이터 준비

In [9]:
fish = pd.read_csv(FILE)

In [13]:
print(fish.shape, fish.columns, sep='\n\n')

(159, 6)

Index(['Species', 'Weight', 'Length', 'Diagonal', 'Height', 'Width'], dtype='object')


In [16]:
# 특성 데이터 + 타겟 분리
# fish_input = fish[fish.columns[1:]].
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target = fish[['Species']].to_numpy()

In [17]:
# 학습용, 테스트용 데이터 분리 => 8:2
train_input, test_input, train_target, test_target = train_test_split(
fish_input, fish_target, random_state=42)

In [18]:
## 표준화 평균 0, 표준편차 1
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)